In [1]:
import numpy as np
from mdp_pokemon_battle import PokemonBattleMDP

# Randomized Game

In [2]:
mdp = PokemonBattleMDP(['Blastoise', 'Charizard', 'Blastoise'], ['Dragonite', 'Blastoise', 'Venusaur', 'Charizard'])

# Initialize variables to track wins, losses, and ties
wins = []
losses = [] 

# Initialize lists to store data for each episode
episode_data = []

# Define the number of episodes to run
num_episodes = 1000

for episode in range(num_episodes):
    state = mdp.reset()
    done = False
    first_move = True
    available_actions = np.array(mdp.get_actions_indexes())
    while not done:
        if first_move == True:
            #action = np.random.randint(len(mdp.ai_team[mdp.current_ai_pokemon].moveset))
            action = np.random.choice(available_actions)
            first_move = False
        else:
            #action = np.random.randint(len(mdp.ai_team[mdp.current_ai_pokemon].moveset))
            action = np.random.choice(available_actions)

        next_state, reward, done = mdp.step(action)

        state = next_state

    # Update win/loss count based on the final reward
    if reward > 1:
        wins.append(1)
        losses.append(0)
    elif reward < -1:
        losses.append(1)
        wins.append(0)

# Print out the distribution of wins, losses, and ties
print("Number of wins:", sum(wins))
print("Number of losses:", sum(losses))

Number of wins: 122
Number of losses: 878


In [259]:
i = 0
j = 0

In [291]:
from IPython.display import display, HTML
from display_battle import *

#mdp = PokemonBattleMDP(['Dragonite', 'Venusaur'], ['Charizard', 'Blastoise', 'Alakazam', 'Charizard', 'Venusaur'])

# episode = episode_data[i]
# if j < len(episode):
#     turn = episode[j]

episode = mdp.episode_history[i]
if j < len(episode):
    step = episode[j]

ai_name = step['ai_team']['full_state']

ai_current_pokemon = step['ai_team']['current_pokemon']
ai_name = mdp.ai_team[ai_current_pokemon].name
ai_pokemon_gif = ai_name.lower() + '-f.gif'

opponent_current_pokemon = step['opponent_team']['current_pokemon']
opponent_name = mdp.opponent_team[opponent_current_pokemon].name
opponent_pokemon_gif = opponent_name.lower() + '-f.gif'


html_ai_gif = '<img src="images/{agent_pokemon_gif}" style="width:10%;">'.format(agent_pokemon_gif = ai_pokemon_gif)
html_opponent_gif = '<img src="images/{opponent_pokemon_gif}" style="width:10%;">'.format(opponent_pokemon_gif = opponent_pokemon_gif)


ai_action = step['ai_team']['action']
ai_hit = step['ai_team']['hit']
ai_effectiveness = step['ai_team']['effectiveness']
ai_hp = step['ai_team']['full_state'][ai_name]['current_hp']
ai_status = step['ai_team']['full_state'][ai_name]['status']

opponent_action = step['opponent_team']['action']
opponent_hit = step['opponent_team']['hit']
opponent_effectiveness = step['opponent_team']['effectiveness']
opponent_hp = step['opponent_team']['full_state'][opponent_name]['current_hp']
opponent_status = step['opponent_team']['full_state'][opponent_name]['status']

# ai_hp, opponent_current_hp, ai_attack = turn

print('Episode:', i+1)
if j < len(episode):
    print('Turn:', j+1)
else:
    print('Turn:', j)
#print()

if j == len(episode):
    if wins[i] == 1:
        display(HTML(html_ai_gif))
        print_ai_turn(ai_name, ai_current_hp=ai_hp, ai_status=ai_status)
        display(HTML(html_opponent_gif))
        print_opponent_turn(opponent_name, opponent_current_hp=0, opponent_status=opponent_status)
        print()
        print(ai_name + " \033[92m" + 'WINS!' + "\033[0m")
    else:
        display(HTML(html_ai_gif))
        print_ai_turn(ai_name, ai_current_hp=0, ai_status=ai_status)
        display(HTML(html_opponent_gif))
        print_opponent_turn(opponent_name, opponent_current_hp=opponent_hp, opponent_status=opponent_status)
        print()
        print(ai_name + " \033[91m" + 'LOSES!' + "\033[0m")
    i += 1
    j = 0
else:
    display(HTML(html_ai_gif))
    print_ai_turn(ai_name, ai_current_hp=ai_hp, ai_status=ai_status)
    display(HTML(html_opponent_gif))
    print_opponent_turn(opponent_name, opponent_current_hp=opponent_hp, opponent_status=opponent_status)
    print()
    print_attack(pokemon_name=ai_name, attack=ai_action, hit=ai_hit, effectiveness=ai_effectiveness)
    print()
    print_attack(pokemon_name=opponent_name, attack=opponent_action, hit=opponent_hit, effectiveness=opponent_effectiveness)
    j += 1

Episode: 5
Turn: 2


Blastoise:
[###################################---------------]
70 / 100
Status:


Dragonite:
[##########################################--------]
85 / 100
Status:

Blastoise uses Ice Beam! It's super effective!

Dragonite uses Dragon Rage! 


# Q-Learning

In [173]:
mdp = PokemonBattleMDP(['Charizard', 'Blastoise', 'Pidgeot', 'Dragonite', 'Rhydon'], 
                       ['Sandslash', 'Exeggutor', 'Ninetales', 'Magneton', 'Vaporeon'])

# Q-learning algorithm
dim_defeated_ai_pokemon = len(mdp.ai_team) + 1
dim_defeated_opponent_pokemon = len(mdp.opponent_team) + 1
dim_current_ai_pokemon = len(mdp.ai_team)
dim_current_opponent_pokemon = len(mdp.opponent_team)
# all_moves = [pokemon.moveset for pokemon in mdp.ai_team]
# dim_actions = sum(len(moves) for moves in all_moves)
dim_actions = len(mdp.all_actions)


Q = np.zeros((dim_defeated_ai_pokemon, 
              dim_defeated_opponent_pokemon, 
              dim_current_ai_pokemon, 
              dim_current_opponent_pokemon, 
              dim_actions))  # Q-table

learning_rate = 0.1
epsilon = 0.1
# gamma = mdp.discount_factor
gamma = 0.9

num_episodes = 1000

# Initialize variables to track wins, losses, and ties
wins = []
losses = [] 

for episode in range(num_episodes):
    state = mdp.reset()
    done = False
    first_move = True
    while not done:
        n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon = state
        available_actions = np.array(mdp.get_actions_indexes())
        if first_move == True:
            # action = np.random.randint(dim_actions)
            action = np.random.choice(available_actions)
            first_move = False
        else:
            if np.random.uniform(0, 1) < epsilon:
                # action = np.random.randint(dim_actions)  # Explore: choose random action
                action = np.random.choice(available_actions)
            else:
                action = np.argmax(Q[n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon][available_actions])
                #print(action)
                action = available_actions[action]
                # print(mdp.ai_team[mdp.current_ai_pokemon].name, ' uses ', mdp.all_unique_moves[action])
                #print(mdp.all_unique_moves[action])

        next_state, reward, done = mdp.step(action)

        next_n_defeated_ai_pokemon, next_n_defeated_opponent_pokemon, next_current_ai_pokemon, next_current_opponent_pokemon = next_state
        # print('State before action:', state)
        # print(action)
        # print('State after action:', next_state)
        Q[n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon, action] += learning_rate * \
            (reward + gamma * np.max(Q[next_n_defeated_ai_pokemon, next_n_defeated_opponent_pokemon, next_current_ai_pokemon, next_current_opponent_pokemon]) - Q[n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon, action])
        state = next_state

        # Update win/loss count based on the final reward
        if reward > 1:
            wins.append(1)
            losses.append(0)
        elif reward < -1:
            losses.append(1)
            wins.append(0)

# Print out the distribution of wins, losses, and ties
print("Number of wins:", sum(wins))
print("Number of losses:", sum(losses))

Number of wins: 918
Number of losses: 82


In [174]:
mdp = PokemonBattleMDP(['Charizard', 'Blastoise', 'Pidgeot', 'Dragonite', 'Rhydon'], 
                       ['Sandslash', 'Exeggutor', 'Ninetales', 'Magneton', 'Vaporeon'])

num_episodes = 1000

# Initialize variables to track wins, losses, and ties
wins = []
losses = [] 

for episode in range(num_episodes):
    state = mdp.reset()
    done = False
    first_move = True
    while not done:
        n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon = state
        available_actions = np.array(mdp.get_actions_indexes())
        # if first_move == True:
        #     action = np.random.choice(available_actions)
        #     first_move = False
        # else:
        #     if np.random.uniform(0, 1) < epsilon:
        #         action = np.random.choice(available_actions) # Explore: choose random action
        #     else:
        action = np.argmax(Q[n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon][available_actions])
        action = available_actions[action]

        next_state, reward, done = mdp.step(action)

        # next_n_defeated_ai_pokemon, next_n_defeated_opponent_pokemon, next_current_ai_pokemon, next_current_opponent_pokemon = next_state
        # Q[n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon, action] += learning_rate * \
        #     (reward + gamma * np.max(Q[next_n_defeated_ai_pokemon, next_n_defeated_opponent_pokemon, next_current_ai_pokemon, next_current_opponent_pokemon]) - Q[n_defeated_ai_pokemon, n_defeated_opponent_pokemon, current_ai_pokemon, current_opponent_pokemon, action])
        state = next_state

        # Update win/loss count based on the final reward
        if reward > 1:
            wins.append(1)
            losses.append(0)
        elif reward < -1:
            losses.append(1)
            wins.append(0)

# Print out the distribution of wins, losses, and ties
print("Number of wins:", sum(wins))
print("Number of losses:", sum(losses))

Number of wins: 1000
Number of losses: 0


In [175]:
i = 0
j = 0

In [255]:
from IPython.display import display, HTML
from display_battle import *

episode = mdp.episode_history[i]
if j < len(episode):
    step = episode[j]

ai_name = step['ai_team']['full_state']

ai_current_pokemon = step['ai_team']['current_pokemon']
ai_name = mdp.ai_team[ai_current_pokemon].name
ai_pokemon_gif = ai_name.lower()

opponent_current_pokemon = step['opponent_team']['current_pokemon']
opponent_name = mdp.opponent_team[opponent_current_pokemon].name
opponent_pokemon_gif = opponent_name.lower()


html_ai_gif = '<img src="images/{agent_pokemon_gif}/{agent_pokemon_gif}.gif" style="width:10%;">'.format(agent_pokemon_gif = ai_pokemon_gif)
html_opponent_gif = '<img src="images/{opponent_pokemon_gif}/{opponent_pokemon_gif}.gif" style="width:10%;">'.format(opponent_pokemon_gif = opponent_pokemon_gif)


ai_action = step['ai_team']['action']
ai_hit = step['ai_team']['hit']
ai_effectiveness = step['ai_team']['effectiveness']
ai_hp = step['ai_team']['full_state'][ai_name]['current_hp']
ai_status = step['ai_team']['full_state'][ai_name]['status']

opponent_action = step['opponent_team']['action']
opponent_hit = step['opponent_team']['hit']
opponent_effectiveness = step['opponent_team']['effectiveness']
opponent_hp = step['opponent_team']['full_state'][opponent_name]['current_hp']
opponent_status = step['opponent_team']['full_state'][opponent_name]['status']

# ai_hp, opponent_current_hp, ai_attack = turn

print('Episode:', i+1)
if j < len(episode):
    print('Turn:', j+1)
else:
    print('Turn:', j)
#print()

if j == len(episode):
    if wins[i] == 1:
        display(HTML(html_ai_gif))
        print_ai_turn(ai_name, ai_current_hp=ai_hp, ai_status=ai_status)
        display(HTML(html_opponent_gif))
        print_opponent_turn(opponent_name, opponent_current_hp=0, opponent_status=opponent_status)
        print()
        print(ai_name + " \033[92m" + 'WINS!' + "\033[0m")
    else:
        display(HTML(html_ai_gif))
        print_ai_turn(ai_name, ai_current_hp=0, ai_status=ai_status)
        display(HTML(html_opponent_gif))
        print_opponent_turn(opponent_name, opponent_current_hp=opponent_hp, opponent_status=opponent_status)
        print()
        print(ai_name + " \033[91m" + 'LOSES!' + "\033[0m")
    i += 1
    j = 0
else:
    display(HTML(html_ai_gif))
    print_ai_turn(ai_name, ai_current_hp=ai_hp, ai_status=ai_status)
    display(HTML(html_opponent_gif))
    print_opponent_turn(opponent_name, opponent_current_hp=opponent_hp, opponent_status=opponent_status)
    print()
    print_attack(pokemon_name=ai_name, attack=ai_action, hit=ai_hit, effectiveness=ai_effectiveness)
    print()
    print_attack(pokemon_name=opponent_name, attack=opponent_action, hit=opponent_hit, effectiveness=opponent_effectiveness)
    j += 1

Episode: 4
Turn: 14


Blastoise:
[############--------------------------------------]
25 / 100
Status: Paralysis


Magneton:
[####################------------------------------]
40 / 100
Status:

Blastoise uses Hydro Pump! 

Magneton uses Thunderbolt! It's super effective!


In [6]:
mdp.episode_history

[[{'ai_pokemon': {'action': 'Wing Attack',
    'max_hp': 100,
    'current_hp': 100,
    'status': None,
    'accuracy': 1,
    'power': 60,
    'hit': 1,
    'effectiveness': 2.0},
   'opponent_pokemon': {'action': 'Bodyslam',
    'max_hp': 100,
    'current_hp': 100,
    'status': None,
    'accuracy': 1,
    'power': 30,
    'hit': 1,
    'effectiveness': 1.0},
   'is_terminal': False,
   'reward': 0},
  {'ai_pokemon': {'action': 'Dragon Rage',
    'max_hp': 100,
    'current_hp': 70,
    'status': None,
    'accuracy': 1,
    'power': 40,
    'hit': 1,
    'effectiveness': 1.0},
   'opponent_pokemon': {'action': 'Synthesis',
    'max_hp': 100,
    'current_hp': 40,
    'status': None,
    'accuracy': 1,
    'power': 0,
    'hit': 1,
    'effectiveness': 0.25},
   'is_terminal': False,
   'reward': 0},
  {'ai_pokemon': {'action': 'Dragon Rage',
    'max_hp': 100,
    'current_hp': 70,
    'status': None,
    'accuracy': 1,
    'power': 40,
    'hit': 1,
    'effectiveness': 1.0},
  